In [1]:
#Подход 1: градиентный бустинг "в лоб"

#1. Считайте таблицу с признаками из файла features.csv с помощью кода, приведенного выше. 
#Удалите признаки, связанные с итогами матча (они помечены в описании данных как отсутствующие в тестовой выборке).
import time
import datetime
import pandas

data = pandas.read_csv('./features.csv', index_col='match_id')

print(data.head())

target = data['radiant_win']
data=data.drop(columns = ['duration','radiant_win','tower_status_radiant','tower_status_dire','barracks_status_radiant','barracks_status_dire'],axis=1)

          start_time  lobby_type  r1_hero  r1_level  r1_xp  r1_gold  r1_lh  \
match_id                                                                     
0         1430198770           7       11         5   2098     1489     20   
1         1430220345           0       42         4   1188     1033      9   
2         1430227081           7       33         4   1319     1270     22   
3         1430263531           1       29         4   1779     1056     14   
4         1430282290           7       13         4   1431     1090      8   

          r1_kills  r1_deaths  r1_items  ...  dire_boots_count  \
match_id                                 ...                     
0                0          0         7  ...                 4   
1                0          1        12  ...                 4   
2                0          0        12  ...                 4   
3                0          0         5  ...                 4   
4                1          0         8  ...             

In [2]:
#2. Проверьте выборку на наличие пропусков с помощью функции count(), которая для каждого столбца показывает число заполненных значений.
#    Много ли пропусков в данных? Запишите названия признаков, имеющих пропуски, и попробуйте для любых двух из них дать обоснование, почему их значения могут быть пропущены.

print(data.count()[data.count()!=data.shape[0]]) 

first_blood_time               77677
first_blood_team               77677
first_blood_player1            77677
first_blood_player2            53243
radiant_bottle_time            81539
radiant_courier_time           96538
radiant_flying_courier_time    69751
radiant_first_ward_time        95394
dire_bottle_time               81087
dire_courier_time              96554
dire_flying_courier_time       71132
dire_first_ward_time           95404
dtype: int64


In [3]:
#3. Замените пропуски на нули с помощью функции fillna(). 
#   На самом деле этот способ является предпочтительным для логистической регрессии,
#    поскольку он позволит пропущенному значению не вносить никакого вклада в предсказание.
#    Для деревьев часто лучшим вариантом оказывается замена пропуска на очень большое или очень маленькое значение
#    — в этом случае при построении разбиения вершины можно будет отправить объекты с пропусками в отдельную ветвь дерева. 
#    Также есть и другие подходы — например, замена пропуска на среднее значение признака.
#    Мы  не требуем этого в задании, но при желании попробуйте разные подходы к обработке пропусков и сравните их между собой."""

data = data.fillna(0)


In [51]:
#4. Какой столбец содержит целевую переменную? Запишите его название.
#   radiant_win

In [5]:
#5.Забудем, что в выборке есть категориальные признаки, и попробуем обучить градиентный бустинг над деревьями
# на имеющейся матрице "объекты-признаки".
# Зафиксируйте генератор разбиений для кросс-валидации по 5 блокам (KFold), не забудьте перемешать при этом выборку (shuffle=True),
# поскольку данные в таблице отсортированы по времени, и без перемешивания можно столкнуться с нежелательными эффектами при оценивании качества.
# Оцените качество градиентного бустинга (GradientBoostingClassifier) с помощью данной кросс-валидации,
# попробуйте при этом разное количество деревьев (как минимум протестируйте следующие значения для количества деревьев: 10, 20, 30).
# Долго ли настраивались классификаторы?
# Достигнут ли оптимум на испытанных значениях параметра n_estimators, или же качество, скорее всего, продолжит расти при дальнейшем его увеличении?

from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingClassifier as GBS 
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score


start_time = datetime.datetime.now()

kf = KFold(n_splits=5, shuffle = True, random_state = 42)  
estimators = [10,20,30,40,50]

for i in estimators:
    gbs = GBS(n_estimators=i, verbose=False, random_state=241)              # Добавить grid_search по learn_rate
    cros_v = cross_val_score(gbs,data, target, cv = kf,scoring='roc_auc')
    #auc = roc_auc_score(target, cros_v)
    print(cros_v.mean())

print ('Time elapsed:', datetime.datetime.now() - start_time)
# Видно, что при значении деревьев больше 30 кривая вышла на плато(изменятются только тысячные доли),
# т.е больше брать смысла нет

D:\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


0.6648506879750012
0.6824618768044435
0.6900064710388155
0.6940387245121103
0.6974943609466162
Time elapsed: 0:05:27.411807


In [6]:
# Подход 2: логистическая регрессия
# Линейные методы работают гораздо быстрее композиций деревьев, поэтому кажется разумным воспользоваться именно ими
# для ускорения анализа данных. Одним из наиболее распространенных методов для классификации является логистическая регрессия.

# Важно: не забывайте, что линейные алгоритмы чувствительны к масштабу признаков!

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import numpy as np

#1. Оцените качество логистической регрессии (sklearn.linear_model.LogisticRegression с L2-регуляризацией) с помощью кросс-валидации
#    по той же схеме, которая использовалась для градиентного бустинга.
#    Подберите при этом лучший параметр регуляризации (C).
#    Какое наилучшее качество у вас получилось?
#    Как оно соотносится с качеством градиентного бустинга?
#    Чем вы можете объяснить эту разницу?
#    Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)


grid = { 'C' : np.power(10.0, np.arange(-5,6))}        # задаём название параметра для поиска максимума и границы сетки
cv = KFold(n_splits = 5, shuffle = True, random_state = 241) 
log_reg = LogisticRegression(penalty = 'l2',  random_state=241)

start_time = datetime.datetime.now()

gs = GridSearchCV(log_reg, grid, scoring = 'roc_auc', cv = cv) # 
gs.fit(data_scaled, target)

print ('Time elapsed:', datetime.datetime.now() - start_time)

for a in gs.grid_scores_:
    print(round(a.mean_validation_score,7),a.parameters, sep = ' ') # — оценка качества по кросс-валидации
   # С_max= 0.01

# Качество логистической регрессии выше, чем у градиентного бустинга, работает она быстрее
# Возможно, на данном этапе это связано с заполнением пропусков, оптимальным для регрессии и масштабированием данных для неё
# Логистическая регрессия даже с данными вида trash-in - trash-out дала преимущество над градиентным бустингом. 
# Если посмотреть честно - мы сталкивали градиентный классификатор с линейным регрессором.
# В документации GBС сказано, что при выборе функции потерь "по умолчанию" он ведёт себя как логистическая регрессия, 
# пусть и с другим методом оптимизации. Таким образом, мы стравливаем нормальную логистическую регрессию, под которую оптимизируем датасет,
#                                                           с костыльно работающим градиентным бустингом. 
# Возможно, в случае более удачного подбора параметров, градиентный бустинг одержал бы верх.

Time elapsed: 0:02:39.609915
0.6951204 {'C': 1e-05}
0.7112501 {'C': 0.0001}
0.7161802 {'C': 0.001}
0.7163415 {'C': 0.01}
0.7163101 {'C': 0.1}
0.7163066 {'C': 1.0}
0.7163063 {'C': 10.0}
0.7163063 {'C': 100.0}
0.7163063 {'C': 1000.0}
0.7163063 {'C': 10000.0}
0.7163063 {'C': 100000.0}


D:\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


In [7]:
#    2. Среди признаков в выборке есть категориальные, которые мы использовали как числовые, что вряд ли является хорошей идеей. 
#        Категориальных признаков в этой задаче одиннадцать: lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, d2_hero, ..., d5_hero.
#        Уберите их из выборки, и проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации.
#        Изменилось ли качество? Чем вы можете это объяснить?

data_drop_cat=data.drop(columns = ['lobby_type','r1_hero','r2_hero','r3_hero','r4_hero','r5_hero','d1_hero','d2_hero','d3_hero','d4_hero','d5_hero'],axis=1)
data_drop_cat_scaled=scaler.fit_transform(data_drop_cat) # изменили раземрность датафрейма, fit заново

start_time = datetime.datetime.now()

gs.fit(data_drop_cat_scaled, target) #gs.fit(data_drop_cat_scaled, target) #


print ('Time elapsed:', datetime.datetime.now() - start_time)
for a in gs.grid_scores_:
    print(round(a.mean_validation_score,7),a.parameters, sep = ' ') # — оценка качества по кросс-валидации
   # С_max= 0.01 

Time elapsed: 0:02:06.716869
0.6950569 {'C': 1e-05}
0.7112484 {'C': 0.0001}
0.7162356 {'C': 0.001}
0.716401 {'C': 0.01}
0.7163738 {'C': 0.1}
0.7163708 {'C': 1.0}
0.7163705 {'C': 10.0}
0.7163705 {'C': 100.0}
0.7163705 {'C': 1000.0}
0.7163705 {'C': 10000.0}
0.7163705 {'C': 100000.0}


D:\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


In [19]:
#3. На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают,
#   какие именно герои играли за каждую команду.
#   Это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, чем другие.
#   Выясните из данных, сколько различных идентификаторов героев существует в данной игре (вам может пригодиться фукнция unique или value_counts).
labels = ['r1_hero','r2_hero','r3_hero','r4_hero','r5_hero','d1_hero','d2_hero','d3_hero','d4_hero','d5_hero']
for i in labels:
    print(len(pandas.unique(data[i])))
N = len(pandas.unique(data[i])) # Возможно, произошла магия, но N = 112. 
                                # При этом всего 108 персонажей

108
108
108
108
108
108
108
108
108
108


In [27]:
#4. Воспользуемся подходом "мешок слов" для кодирования информации о героях.
#    Пусть всего в игре имеет N различных героев.
#    Сформируем N признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче;
#    единице, если i-й герой играл за команду Radiant;
#    минус единице, если i-й герой играл за команду Dire.
#    Ниже вы можете найти код, который выполняет данной преобразование.
#    Добавьте полученные признаки к числовым, которые вы использовали во втором пункте данного этапа.


X_pick = np.zeros((data.shape[0], N))

for i, match_id in enumerate(data.index):
    for p in range(5):
        X_pick[i, data.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, data.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  from ipykernel import kernelapp as app


       0    1    2    3    4    5    6    7    8    9    ...  102  103  104  \
0      0.0  0.0  0.0 -1.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  1.0   
1      0.0  0.0  0.0  0.0  0.0  0.0 -1.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2      0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
3      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
4      0.0  0.0  0.0  0.0  0.0  0.0  0.0 -1.0  0.0  0.0  ...  0.0  0.0  0.0   
...    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
97225  1.0  0.0 -1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
97226  0.0  0.0  0.0  1.0 -1.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
97227  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
97228  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 -1.0  0.0  ...  0.0  0.0  0.0   
97229  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   

       105  106  107  108  109  110  111  
0      0

In [35]:
X_pick_df = pandas.DataFrame(X_pick)

#print(X_pick_df) # Видим, что автоматически нумерация стартует с 0. Нумерация персонажей начинается с единицы, учтём это:

X_pick_df.columns = range(1,N+1)

#print(X_pick_df)

cols = [col for col in X_pick_df.columns if col in heroes] # Всего 112 столбцов и 108 героев. Какие-то лишние. Уберём их.

X_pick_df = X_pick_df[cols] 

#print(X_pick_df)

data_drop_cat_scaled = pandas.DataFrame(data_drop_cat_scaled)
data_bag_words = pandas.concat([data_drop_cat_scaled, X_pick_df], axis = 1) # Горизонтальная склейка

In [36]:
#5.    Проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации.
#     Какое получилось качество?
#     Улучшилось ли оно?
#     Чем вы можете это объяснить?


start_time = datetime.datetime.now()
 
gs.fit(data_bag_words, target)

print ('Time elapsed:', datetime.datetime.now() - start_time)

for a in gs.grid_scores_:
    print(round(a.mean_validation_score,7),a.parameters, sep = ' ') # — оценка качества по кросс-валидации
   # С_max= 0.1 Таки изменился. Не зря просили ещё раз gridsearch сделать

#   Качество модели значительно возросло при добавлении категориальных признаков в удобоваримом виде,
#                                                                         что свидетельствует об их важности
#                               Т.е от выбранных персонажей во многом зависит исход сражения

Time elapsed: 0:04:22.501979
0.6991713 {'C': 1e-05}
0.7250221 {'C': 0.0001}
0.7462962 {'C': 0.001}
0.751736 {'C': 0.01}
0.7519374 {'C': 0.1}
0.7519196 {'C': 1.0}
0.7519174 {'C': 10.0}
0.751917 {'C': 100.0}
0.751917 {'C': 1000.0}
0.751917 {'C': 10000.0}
0.7519172 {'C': 100000.0}


D:\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


In [59]:
#6.  Постройте предсказания вероятностей победы команды Radiant для тестовой выборки с помощью лучшей из изученных моделей
#    (лучшей с точки зрения AUC-ROC на кросс-валидации).
#    Убедитесь, что предсказанные вероятности адекватные — находятся на отрезке [0, 1], не совпадают между собой
#    (т.е. что модель не получилась константной).

test = pandas.read_csv('./features_test.csv', index_col='match_id')
#print(test)
test = test.fillna(0)
test_drop_cat=test.drop(columns = ['lobby_type','r1_hero','r2_hero','r3_hero','r4_hero','r5_hero','d1_hero','d2_hero','d3_hero','d4_hero','d5_hero'],axis=1)
test_drop_cat_scaled = scaler.transform(test_drop_cat) # Есть уже обученный scaler для дропнутых категориальных признаков
test_drop_cat_scaled = pandas.DataFrame(test_drop_cat_scaled)
test_pick = np.zeros((test.shape[0], N))

for i, match_id in enumerate(test.index):
    for p in range(5):
        test_pick[i, test.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        test_pick[i, test.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

test_pick_df = pandas.DataFrame(test_pick)
test_pick_df.columns = range(1,N+1)
cols = [col for col in test_pick_df.columns if col in heroes]
test_pick_df = test_pick_df[cols] 
test_drop_cat_scaled = pandas.DataFrame(test_drop_cat_scaled)
test_bag_words = pandas.concat([test_drop_cat_scaled, test_pick_df], axis = 1)
pred = gs.predict_proba(test_bag_words)[:, 1] 

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  from ipykernel import kernelapp as app
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  app.launch_new_instance()


In [63]:
print(max(pred), min(pred), pred.mean())
#Получили адекватно выглядещее распределение вероятности - максимально возможное - близко к 1, минимальное - почти 0, среднее 1/2
# Распределение типа "Колокол". Возможно, и гауссово, но доказывать это не будем :)

0.9963714245178639 0.008428978265274441 0.5170234016735038
